## Support Vector Machines

In [4]:
# Linear SVM : SVC
from sklearn.svm import LinearSVC
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)] # petal length, petal width
y = (iris["target"] == 2).astype(np.float64) # Iris virginica

svm_clf = Pipeline([
    ("scaling", StandardScaler()),
    ("svm", LinearSVC(C = 1, loss = "hinge"))
])
svm_clf.fit(X, y)
svm_clf.predict([[5.5, 1.7]])

<IPython.core.display.Javascript object>

array([1.])

In [5]:
# Non-Linear data
# Polynomial Features with Linear SVC

from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
X, y = make_moons(n_samples=100, noise=0.15)

poly_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree = 3)),
    ("scaling", StandardScaler()),
    ("svm", LinearSVC(C = 10, loss = "hinge"))
])

poly_svm_clf.fit(X, y)
poly_svm_clf.predict([[5.5, 1.7]])

array([1], dtype=int64)

In [6]:
# Non-Linear data
# Kernel Trick with SVC : Avoids exploding of features with PolyReg

from sklearn.svm import SVC
X, y = make_moons(n_samples=100, noise=0.15)

poly_kernel_svm_clf = Pipeline([
        ("scaling", StandardScaler()),
    ("svm", SVC(kernel = 'poly', degree = 3, coef0 = 1, C = 5))
])

poly_kernel_svm_clf.fit(X, y)
poly_kernel_svm_clf.predict([[5.5, 1.7]])

array([1], dtype=int64)

In [8]:
# Support Vector Regression
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon = 1.5)
svm_reg.fit(X, y)

LinearSVR(epsilon=1.5)

In [9]:
# Poly SVR
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel = 'poly', degree = 2, C = 0.1, epsilon = 0.1)
svm_poly_reg.fit(X, y)

SVR(C=0.1, degree=2, kernel='poly')